In [1]:
import pandas as pd
import numpy as np

In [118]:
engagements = pd.read_csv('daily_engagement.csv')
enrollments = pd.read_csv('enrollments.csv')
submissions = pd.read_csv('project_submissions.csv')

# Remove test accounts
enrollments = enrollments[enrollments['is_udacity'] == False]

In [119]:
engagements_sum = engagements.groupby('account_key').sum()
engagements_sum.columns = ['e_sum_' + str(i) for i in engagements_sum.columns]
engagements_count = engagements.groupby('account_key').count()
engagements_count.columns = ['e_count_' + str(i) for i in engagements_count.columns]

In [125]:
engagements_sum = engagements_sum.reset_index()
engagements_count = engagements_count.reset_index()

In [127]:
enrollments = enrollments.merge(engagements_sum, how='outer', on='account_key')

In [128]:
enrollments = enrollments.merge(engagements_count, how='outer', on='account_key')

In [129]:
submissions = submissions.merge(enrollments, how='inner', on='account_key')

data = submissions.copy()
data = data[data.lesson_key == 3176718735]
for i in data.drop('assigned_rating', axis=1).columns:
    #if 'e_' not in i: del data[i]
    if 'projects_com' in i: del data[i]
    elif data[i].nunique() == 1: del data[i]

In [130]:
y = np.where((data['assigned_rating'] == 'PASSED') |\
             (data['assigned_rating'] == 'DISTINCTION'),
              1, 0)
x = data.select_dtypes('number')

In [131]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

xg = XGBRegressor(max_depth=2)

cv = cross_val_score(xg, x, y, scoring='roc_auc', cv=4
                    )

cv.mean(), cv.std()

(0.55806347754029817, 0.0096604869025730417)

In [132]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

xg = LogisticRegression()

cv = cross_val_score(xg, x.fillna(x.median()), y, scoring='roc_auc', cv=4)

cv.mean(), cv.std()

(0.54814087230617903, 0.020394817035493781)

In [117]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

xg = DecisionTreeClassifier(max_depth=3)

cv = cross_val_score(xg, x.fillna(x.median()), y, scoring='roc_auc', cv=4)

cv.mean(), cv.std()

(0.57255552169228241, 0.012475391879279093)

In [133]:
data.shape

(1941, 17)

In [ ]:
enrollments